# Lab2 - TSP


In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from icecream import ic
import random
import math

logging.basicConfig(level=logging.DEBUG)

In [ ]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX= np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

In [ ]:
def tsp_cost(tsp):
    tot_cost =  0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

#


## Greedy Algorithm (fast but approximate)

In [ ]:
visited = np.full(len(CITIES), False) # all cities are not visited
dist = DIST_MATRIX.copy()
city = 0 # start from city 0
visited[city] = True # mark city 0 as visited
tsp = list()
tsp.append(int(city)) # add city 0 to the list
while not np.all(visited): # while there are cities not visited
    dist[:, city] = np.inf
    closest = np.argmin(dist[city]) # find the closest city
    visited[closest] = True # mark the closest city as visited
    city = closest # move to the closest city
    tsp.append(int(city)) # add the closest city to the list

tsp.append(tsp[0]) # return to the starting city
tot_cost = 0 # calculate the total cost of the path
for c1, c2 in zip(tsp, tsp[1:]): # for each pair of consecutive cities
    tot_cost += DIST_MATRIX[c1, c2] # add the distance between the cities

logging.info(f"Cost: {tsp_cost(tsp):.2f}km") 
logging.info(f"Steps: {len(tsp)-1}")


#

## Evolutionary algorithm (slower yet more accurate)


In [ ]:
# Parameters
NUM_CITIES = len(CITIES)
POP_SIZE = 100
GENERATIONS = round(50000/NUM_CITIES)
MUTATION_RATE = 0.1
ELITE_RATE = 0.1

# Initialize population from greedy solution
def initialize_population():
    population = [tsp for _ in range(int(POP_SIZE))] # initialize population with greedy solution
    return population 

# Fitness function
def fitness(individual):
    return 1 / tsp_cost(individual) # the higher the fitness, the lower the cost

# Selection with elitism
def selection(population, fitness_scores):
    elite_count = int(POP_SIZE * ELITE_RATE) # number of elite individuals
    elite_individuals = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)[:elite_count] # select elite individuals
    elite_population = [ind for ind, _ in elite_individuals]
    
    remaining_population = random.choices(population, weights=fitness_scores, k=POP_SIZE - elite_count) # select remaining individuals
    return elite_population + remaining_population

# Ordered crossover
def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(NUM_CITIES), 2)) # select two random indices
    child = [-1] * NUM_CITIES # initialize child with -1
    child[start:end] = parent1[start:end] # copy the selected slice from parent1 to child

    pointer = 0 
    for gene in parent2: # iterate over parent2
        if gene not in child: # if the gene is not already in the child
            while child[pointer] != -1: # find the next empty slot
                pointer += 1 
            child[pointer] = gene # copy the gene to the empty slot
    return child

# Mutation
def mutation(individual):    
    one, two = sorted(random.sample(range(NUM_CITIES), 2)) # select two random indices
    new_individual = individual[:] # copy the individual
    new_individual[one] = individual[two] 
    new_individual[two] = individual[one] # swap the two cities
    if(fitness(new_individual) > fitness(individual)):
        return new_individual # return the new individual if it has better fitness
    return individual

# Evolution process
def evolve_population(population):
    fitness_scores = [fitness(ind) for ind in population] # calculate fitness scores
    new_population = [] # initialize new population
    
    selected_individuals = selection(population, fitness_scores) # select individuals for the next generation
    
    while len(new_population) < POP_SIZE: # generate new population
        parent1, parent2 = random.sample(selected_individuals, 2) # select two random parents
        child = crossover(parent1, parent2) # generate child from parents
        child = mutation(child) if random.random() < MUTATION_RATE else child # apply mutation
        new_population.append(child) # add child to new population
    
    return new_population

# Run the algorithm
population = initialize_population() # initialize population
best_route = None # initialize best route
best_fitness = 0 # initialize best fitness

for generation in range(GENERATIONS): 
    population = evolve_population(population) # evolve population
    fitness_scores = [fitness(ind) for ind in population] # calculate fitness scores
    
    current_best_idx = np.argmax(fitness_scores) # find the best indiviual
    current_best_fitness = fitness_scores[current_best_idx] # find the best fitness
    
    if current_best_fitness > best_fitness: # if the best individual is better than the previous best
        best_fitness = current_best_fitness # update the best fitness
        best_route = population[current_best_idx] # update the best route   

best_route = np.append(best_route, best_route[0])


logging.info(f"Cost: {1 / best_fitness:.2f}km") 
logging.info(f"Steps: {GENERATIONS}")

